In [1]:
# importing necessary libs

import numpy as np
from scipy.optimize import minimize

In [2]:
# Constants and properties

k_water = 0.6 # Thermal conductivity of water (W/m*K)
mu_water = 0.001 # Dynamic viscosity of water at room temperature (Pa*s)
cp_water = 4181 # Specific heat capacity of water (kg/m^3)
density_water = 1000 # Density of water (kg/m^3)
cost_per_volume = 10 # Cost per unit volume of material (dollars per cubic meter)

In [3]:
# Objective function to minimize

def objective(x):
    outer_diameter, wall_thickness, flow_rate, number_of_tubes = x
    inner_diameter = outer_diameter - 2*wall_thickness
    cross_section_area = np.pi * (inner_diameter**2) / 4
    velocity = flow_rate / (cross_section_area * number_of_tubes)
    reynolds_number = (inner_diameter * velocity * density_water) / mu_water
    
    # Nusselt number correlations (Dittus-Boelter equation for turbulent flow)
    if reynolds_number > 4000:
        nusselt_number = 0.023 * (reynolds_number ** 0.8) * (cp_water * mu_water / k_water) ** 0.4
    else:
        nusselt_number = 4.36 # Laminar flow approximation
        
    heat_transfer_coefficient = nusselt_number * k_water / inner_diameter
    total_heat_transfer_area = np.pi * inner_diameter * number_of_tubes
    heat_transfer_rate = heat_transfer_coefficient * total_heat_transfer_area * 10 # Assuming a 10 degree celsius temperature difference
    
    pressure_drop = 0.08 * (density_water / 2) * velocity**2 * (outer_diameter / inner_diameter) * number_of_tubes
    
    material_volume = np.pi * (outer_diameter**2 - inner_diameter**2) * 1 * wall_thickness * number_of_tubes
    material_cost = material_volume * cost_per_volume
    
    return -heat_transfer_rate + 0.1 * pressure_drop + 0.01 * material_cost

In [4]:
# Constraints

def min_thickness_constraint(x):
    return x[1] - 0.005 # Minimum wall thickness of 0.005 meters

def flow_rate_constraint(x):
    return x[2] - 0.01 # Minimum flow rate of 0.01 m^3/s

def number_of_tubes_constraint(x):
    return x[3] - 1 # At least one tube must be used

In [5]:
# initial guesses

x0 = [0.05, 0.005, 0.1, 10]

In [6]:
# Define bounds

bounds = [(0.05, 0.2), (0.005, 0.02), (0.01, 2.0), (1, 100)]

In [7]:
# Combining all constraints

constraints = [{'type': 'ineq', 'fun': min_thickness_constraint},
               {'type': 'ineq', 'fun': flow_rate_constraint},
               {'type': 'ineq', 'fun': number_of_tubes_constraint}]

In [8]:
# Perform the optimization

solution = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=constraints)

In [9]:
# Output results

print('Optimal outer diameter (m):', solution.x[0])
print('Optimal wall thickness (m):', solution.x[1])
print('Optimal flow rate (m^3/s):', solution.x[2])
print('Optimal number of tubes:', int(solution.x[3]))
print('Objective function value:', solution.fun)

Optimal outer diameter (m): 0.05
Optimal wall thickness (m): 0.005
Optimal flow rate (m^3/s): 0.1
Optimal number of tubes: 10
Objective function value: -234824.76448735717
